In [ ]:
整体原理是
#define REGISTER_ONNXRUNTIME_OPS(domain, name)     \
static char __domain_##domain##name[] = #domain; \
static OrtOpsRegistry<__domain_##domain##name, name> ort_ops_registry_##domain##name {}

用一个宏，再每个自定义操作cpp中将REGISTER_ONNXRUNTIME_OPS(mmdeploy, GridSampleOp)加入，
作用就是，在全局中创建一个字典{domian:[op1,op2,op3]}，直接把customop放到字典中，
之后实现RegisterCustomOps，将字典中所有的自定义操作全部注册到系统中

In [ ]:
customOP继承的父类为Ort::CustomOpBase，只是一个空类
里面有11个简单的函数，都是用lam方式实现的，所以当自定义操作符继承他后，直接按照简单执行即可重写
没有什么好解读的

In [ ]:
opkernel:(要实现默认构造和compute函数)
    构造函数：
        传入两个参数，一个是api函数包类OrtApi，里面有很多函数；一个是参数集合OrtKernelInfo
        用api从参数集合中解析出参数，赋值给kernel的函数
        api本身也是一个参数，
        例子：ort_.KernelInfoGetAttribute<int64_t>(info, "align_corners");读取align_corners参数
        
        allocator_ = Ort::AllocatorWithDefaultOptions();一般还会创建一个分配器，可能后续会用
    Compute函数：
        传入一个参数OrtKernelContext
        
        1、先读取kernel内中所有的参数
        
        2、获取每个输入，并且转化为指针，可以将context里的输入当作列表处理
        const OrtValue *offset = ort_.KernelContext_GetInput(context, 1);
        const float *offset_data = reinterpret_cast<const float *>(ort_.GetTensorData<float>(offset));
        取出形状
        OrtTensorTypeAndShapeInfo* info = ort.GetTensorTypeAndShape(offset)
        
        
        3、计算
        
        4、设置输出空间，进行填充就行了
        
        OrtValue *output =
        ort_.KernelContext_GetOutput(context, 0, output_dims.data(), output_dims.size());
        float *out_ptr = ort_.GetTensorMutableData<float>(output);